In [1]:
import cv2 as cv
from PIL import Image
import time
import os

In [3]:
DIR_CROPPED = r"C:\Users\Anna\Documents\siamese_face_recognition\data"

In [4]:
face_cascade = cv.CascadeClassifier(r'C:\Users\Anna\Documents\siamese_face_recognition\data\face_detection_default.xml')
eye_cascade = cv.CascadeClassifier(r'C:\Users\Anna\Documents\siamese_face_recognition\data\eye_detection.xml')
smile_cascade = cv.CascadeClassifier(r'C:\Users\Anna\Documents\siamese_face_recognition\data\smile_detection.xml')

In [5]:
def crop(img, x, y, w, h, increase):
    """crops an image to a certain size

    Args:
      img: image
      x, y, w, h: coordiantes, width, hight
      increase: scale
    """
    w=h=int(min(img.shape[0],img.shape[1] ,int(w*increase),int(h*increase))//increase)
    
    new_x=max(int(x-w*(increase-1)//2),0)
    new_y=max(int(y-h*(increase-1)//2),0)

    new_x=new_x-max(int(new_x+increase*w)-img.shape[1],0)
    new_y=new_y-max(int(new_y+increase*h)-img.shape[0],0)

    x=new_x
    y=new_y

    crop_img = img[y:y+int(h*increase), x:x+int(w*increase)]

    return crop_img

In [6]:
def get_name(DIR):
    """counts the number of images in directory and return a number used as a name of a new image 

    Args:
      DIR: image directory
    """
    d = os.listdir(DIR)
    if len(d) == 0:
        return "0"
    x = [s.split('.')[0] for s in d]
    x.sort()
    return str(int(x[-1]) + 1)

In [5]:
def dataset_creation(name, DIR_CROPPED, dataset = 'train'):
    """adds images to a specific directory under a specific name

    Args:
      name: name of a person
      DIR_CROPPED: directory to add
      dataset: {train, test, validation}
    """
    DIR_CROPPED = os.path.join(DIR_CROPPED, dataset)
    DIR_CROPPED = os.path.join(DIR_CROPPED, name)
    
    # create dir if not exist    
    if not os.path.exists(DIR_CROPPED):
        os.makedirs(DIR_CROPPED)

    time = 20
    cap = cv.VideoCapture(0)
    while True:
        # Read the frame
        _, img = cap.read()
        img =cv.flip(img, 1)
        img_rec = img.copy()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.1, 10)
        eyes = eye_cascade.detectMultiScale(gray, 1.3, 15)

        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            cv.rectangle(img_rec, (x, y), (x+w, y+h), (255, 0, 0), 2)
        for (x, y, w, h) in eyes:
            cv.rectangle(img_rec, (x, y), (x+w, y+h), (0, 255, 0), 2)

        if len(faces) >= 1:
            if len(eyes) == len(faces)*2:
                time -= 1
            else:
                time = 20

            if time == 0:               
                for (x, y, w, h) in faces:
                    img_cropped = crop(img, x, y, w, h, 1.6)   
                    path = DIR_CROPPED + "\\" + get_name(DIR_CROPPED) + ".jpg"
                    cv.imwrite(path, img_cropped)
                    cv.imshow('Face', img_cropped)
                    while True:
                        if cv.waitKey(5) & 0xFF == ord('x'):
                            cv.destroyWindow('Face')
                            break

        # Display
        cv.imshow('Capture', img_rec)                

        # Stop if escape key is pressed            
        if cv.waitKey(1) & 0xFF == ord('x'):
            cv.destroyAllWindows()
            break


    # Release the VideoCapture object
    cap.release() 

In [6]:
dataset_creation('Anna2', DIR_CROPPED, 'train')